In [30]:
import pandas as pd
import numpy as np

In [33]:
gpd.read_file('data/microwave/microwave.gpkg')

,price,title,added_time,metro,seller_name,seller_rating,link,parsed_at,address,description,dist_to_metro,seller_review,seller_status,views,geometry
0,1800.0,"Доставка свч печей с гарантией Lg 658, Sanyo 7...",2020-03-26T13:18:00,Проспект Ветеранов,Доставка б.у. СВЧ с гарантией,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T02:49:16,"Санкт-Петербург, пр-т Ветеранов, 24",ПРЕДОПЛАТА ПО КАРТЕ- ПРИЗНАК МОШЕННИЧЕСТВАПриё...,400.0,27.0,Компания,1508.0,None
1,2990.0,Микроволновая печь,2020-03-25T10:47:00,Волковская,Анатолий,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T02:49:18,"Санкт-Петербург, ул. Салова, 27АД",Микроволновая печь LG Мощность 700 ВтОбъём 19 ...,800.0,0.0,Частное лицо,33.0,None
2,500.0,Микроволновая печь SAMSUNG. Микроволновка,2020-03-31T02:05:00,Ладожская,Саша,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T02:49:20,"Санкт-Петербург, Заневский пр-т, 69",Крутая рабочая микроволновка. Всё работает. Пр...,NaN,0.0,Частное лицо,1.0,None
3,1500.0,Moulinex Faciclic glass мотор,2020-03-31T00:56:00,Чернышевская,Марина,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T02:49:21,Санкт-Петербург,Продаю мотор с ножом и крышку от блендера Moul...,NaN,1.0,Частное лицо,1.0,None
4,2000.0,Микроволновая печь Supra,2020-03-31T00:50:00,Площадь Мужества,1,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T02:49:23,"Санкт-Петербург, Кушелевская дорога",Хорошая и чистая!! микроволновка в отличном со...,1200.0,0.0,Компания,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,2300.0,Микроволновая печь,2020-03-01T10:55:00,Купчино,андрей,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T03:40:32,"Санкт-Петербург, Купчинская ул., 9к1","Небольшая, чистая, довольно мощная микроволнов...",1500.0,4.0,Частное лицо,1827.0,None
1258,2000.0,Микроволновая печь Panasonic,2020-03-01T10:15:00,Международная,Диана,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T03:40:34,Санкт-Петербург,Все работает .,NaN,1.0,Частное лицо,255.0,None
1259,2500.0,Микроволновка Panasonic,2020-03-01T09:51:00,Горьковская,Сергей,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T03:40:36,"Санкт-Петербург, ул. Куйбышева, 9/8",В хорошем состоянии класса А,500.0,1.0,Компания,165.0,None
1260,4000.0,Микровалновая печь Lg(mс194a.800vт.без гриля),2020-03-01T09:44:00,Московская,Андрей,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31T03:40:38,"Санкт-Петербург, 5-й Предпортовый пр., 4к3",Продам микровалновку идиальное состояние!!!(cм...,1100.0,2.0,Частное лицо,614.0,None


In [32]:
df = pd.read_csv('data/microwave/microwave.csv')
df.head()

,price,title,added_time,metro,seller_name,seller_rating,link,parsed_at,address,description,dist_to_metro,seller_review,seller_status,views
0,1800.0,"Доставка свч печей с гарантией Lg 658, Sanyo 7...",26 марта в 13:18,Проспект Ветеранов,Доставка б.у. СВЧ с гарантией,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31 02:49:16.938293,"Санкт-Петербург, пр-т Ветеранов, 24",ПРЕДОПЛАТА ПО КАРТЕ- ПРИЗНАК МОШЕННИЧЕСТВАПриё...,400 м,27.0,Компания,1508.0
1,2990.0,Микроволновая печь,25 марта в 10:47,Волковская,Анатолий,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31 02:49:18.636017,"Санкт-Петербург, ул. Салова, 27АД",Микроволновая печь LG Мощность 700 ВтОбъём 19 ...,800 м,0.0,Частное лицо,33.0
2,500.0,Микроволновая печь SAMSUNG. Микроволновка,сегодня в 02:05,Ладожская,Саша,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31 02:49:20.241951,"Санкт-Петербург, Заневский пр-т, 69",Крутая рабочая микроволновка. Всё работает. Пр...,NaN,0.0,Частное лицо,1.0
3,1500.0,Moulinex Faciclic glass мотор,сегодня в 00:56,Чернышевская,Марина,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31 02:49:21.845578,Санкт-Петербург,Продаю мотор с ножом и крышку от блендера Moul...,NaN,1.0,Частное лицо,1.0
4,2000.0,Микроволновая печь Supra,сегодня в 00:50,Площадь Мужества,1,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,2020-03-31 02:49:23.108644,"Санкт-Петербург, Кушелевская дорога",Хорошая и чистая!! микроволновка в отличном со...,"1,2 км",0.0,Компания,1.0


Переведем столбец ```parsed_at``` в *datetime-object*

In [24]:
df['parsed_at'] = pd.to_datetime(df['parsed_at'])
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   price          1309 non-null   object        
 1   title          1309 non-null   object        
 2   added_time     1309 non-null   object        
 3   metro          1309 non-null   object        
 4   seller_name    1309 non-null   object        
 5   seller_rating  1309 non-null   object        
 6   link           1309 non-null   object        
 7   parsed_at      1309 non-null   datetime64[ns]
 8   address        1309 non-null   object        
 9   description    1309 non-null   object        
 10  dist_to_metro  1309 non-null   object        
 11  seller_review  1309 non-null   object        
 12  seller_status  1309 non-null   object        
 13  views          1309 non-null   object        
 14  geometry       0 non-null      geometry      
dtypes: datetime64

# Посмотрим на NaN

In [25]:
df.isna().sum()

price               0
title               0
added_time          0
metro               0
seller_name         0
seller_rating       0
link                0
parsed_at           0
address             0
description         0
dist_to_metro       0
seller_review       0
seller_status       0
views               0
geometry         1309
dtype: int64

Удалим все записи, в которых нет цены

In [6]:
df = df[df['price'].notna()]

In [7]:
df.isna().sum()

price              0
title              0
added_time         0
metro            164
seller_name        0
seller_rating    519
link               0
parsed_at          0
address            0
description        0
dist_to_metro    384
seller_review      0
seller_status      0
views              0
dtype: int64

Теперь NaN-ов в важных столбцах нет. Ближайшее метро и расстояние до него можно будет получить через API Яндекса, а на рейтинг продавца можно не обращать особого внимания.

# Избавимся от объявлений, продающих неисправные холодильники или запчасти для них 
Профильтруем датафрейм по списку слов в заголовке. Потом можно будет сделать более продвинутый алгоритм, который будет работать с описанием товара.  
Функция будет принимать текстовый файл со списком запрещенных слов.

In [8]:
def filter_df(df, filter_file):
    with open(filter_file, 'r', encoding='utf-8') as f:
        filter_words = f.readlines()
    return df[~df.title.str.lower().str.contains('|'.join(filter_words))]

In [9]:
df = filter_df(df, 'data/fridge/fridge_filter.txt')
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
price,1256.0,1680.601115,1242.546649,1.0,500.0,1500.0,2612.50,4500.0
seller_rating,737.0,4.626052,0.884759,1.0,4.8,5.0,5.00,5.0
seller_review,1256.0,4.528662,9.114192,0.0,0.0,1.0,5.00,128.0
views,1256.0,604.907643,1835.714697,1.0,45.0,159.0,523.25,34730.0


# Переведем строки в более удобные форматы
## Переведем столбец *dist_to_metro*

In [10]:
def dist_to_float(x):
    if type(x) != str:
        return x
    dist, unit = x.split()
    dist = dist.replace(',', '.')
    dist = float(dist)
    if unit == 'км':
        dist *= 1000
    return dist
df['dist_to_metro'] = df['dist_to_metro'].map(dist_to_float)

In [11]:
df.dist_to_metro.describe()

count     872.000000
mean     1207.683486
std      1094.773368
min       100.000000
25%       500.000000
50%       900.000000
75%      1500.000000
max      8500.000000
Name: dist_to_metro, dtype: float64

## Переведем столбец *added_time*

In [12]:
df.added_time.str.contains(' в ').value_counts()

True    1256
Name: added_time, dtype: int64

Каждая запись имеет ```' в '``` внутри. Значит, мы сможем безопасно разделить стринг по этому токену

In [13]:
from datetime import datetime, timedelta

def str_to_datetime(string_date, parsed_at):
    date, time = string_date.split(' в ')
    hour, minute = list(map(int, time.split(':')))
    if date == 'вчера':
        yesterday = parsed_at - timedelta(days=1)
        day = yesterday.day
        month = yesterday.month
    elif date == 'сегодня':
        day = parsed_at.day
        month = parsed_at.month
    else:
        ru_to_num = {
            'января':1,
            'февраля':2,
            'марта':3,
            'апреля':4,
            'мая':5,
            'июня':6,
            'июля':7,
            'августа':8,
            'сентября':9,
            'октября':10,
            'ноября':11,
            'декабря':12,
        }
        day, month = date.split()
        day = int(day)
        month = ru_to_num[month]
    return datetime(2020, month, day, hour, minute) 

In [14]:
df['added_time'] = df.apply(lambda x: str_to_datetime(x['added_time'], x['parsed_at']), axis=1)

In [15]:
df.added_time

0      2020-03-26 13:18:00
1      2020-03-25 10:47:00
2      2020-03-31 02:05:00
3      2020-03-31 00:56:00
4      2020-03-31 00:50:00
               ...        
1304   2020-03-01 10:55:00
1305   2020-03-01 10:15:00
1306   2020-03-01 09:51:00
1307   2020-03-01 09:44:00
1308   2020-03-31 03:17:00
Name: added_time, Length: 1256, dtype: datetime64[ns]

# Получим координаты места по адресу через Геокодер Яндекса

In [15]:
import requests
import json
from secret import API_KEY
example = df.iloc[2].address
example

'Санкт-Петербург, Калининский район, муниципальный округ Академическое'

In [16]:
request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, example.replace(' ', '+'))
r = requests.get(request)
response = json.loads(r.text)
response

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'request': 'Санкт-Петербург, Калининский район, муниципальный округ Академическое',
     'results': '10',
     'found': '1'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'other',
        'text': 'Россия, Санкт-Петербург, Калининский район, муниципальный округ Академическое',
        'kind': 'district',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Санкт-Петербург, Калининский район, муниципальный округ Академическое',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Северо-Западный федеральный округ'},
          {'kind': 'province', 'name': 'Санкт-Петербург'},
          {'kind': 'locality', 'name': 'Санкт-Петербург'},
          {'kind': 'district', 'name': 'Калининский район'},
          {'kind': 'district', 'name': 'муниципальный округ Академическое'}]},
        'AddressD

In [17]:
def get_coords(x):
    request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, x.replace(' ', '+'))
    r = requests.get(request)
    coords = json.loads(r.text)['response']
    coords = coords['GeoObjectCollection']['featureMember'][0]
    coords = coords['GeoObject']['Point']['pos']
    coords = coords.split()[::-1]
    coords = list(map(float, coords))
    return coords

In [18]:
get_coords(example)

[60.01691, 30.385721]

In [19]:
df['coords'] = df.address.map(get_coords)

"Растащим" список на две отдельные колонки

In [20]:
df[['lat', 'long']] = pd.DataFrame(df.coords.values.tolist(), index=df.index)
df = df.drop('coords', 1)

Соберем это в отдельную функцию

In [21]:
def geocode(df):
    def get_coords(x):
        request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, x.replace(' ', '+'))
        r = requests.get(request)
        coords = json.loads(r.text)['response']
        coords = coords['GeoObjectCollection']['featureMember'][0]
        coords = coords['GeoObject']['Point']['pos']
        coords = coords.split()[::-1]
        coords = list(map(float, coords))
        return coords
    
    df['coords'] = df.address.map(get_coords)
    df[['lat', 'long']] = pd.DataFrame(df.coords.values.tolist(), index=df.index)
    df = df.drop('coords', 1)
    return df